# Optimizing Performance (Single-Core)

At the heart of fast parallel code must be fast serial code. Parallelism can make a good serial code faster. But it can also make a bad code even worse. One can write terribly slow code in any language, including Julia. In this notebook we want to understand what makes Julia code slow and how to detect and avoid common pitfalls. This will lead to multiple concrete performance tips that will help you speed up your Julia code and to write more efficient code in the first place.

By far the most common reasons for slow Julia code are

* **break-down of type inference** ("type instabilities")
* **(unnecessary) allocations**

Once you have those under control you might want to care about

* **memory access optimizations** (spatial and temporal locality)
* **SIMD** (single-instruction multiple data)
* etc.

## Type inference

* **Type stability**: A function `f` is type stable if for a given set of input argument types the return type is always the same. In particular, it means that the type of the output of `f` cannot vary depending on the **values** of the inputs.


* **Type instability**: The return type of a function `f` is not predictable just from the type of the input arguments alone.

Instructive example: `f() = rand((1.23, 100, 1f0, "string"))`

More loosely speaking:
* type instability == `Any`'s appearing unexpectedly in `@code_warntype` output.

In [1]:
f() = rand((1.23, 100, 1f0, "string"))

f (generic function with 1 method)

In [2]:
f()

1.23

In [3]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#

::Core.Const(f)
Body::Any
1 ─

 %1 = Core

.tuple(1.23, 100, 1.0f0, "string")

::Core.Const((1.23, 100, 1.0f0, "string"))
│   %2 = Main.rand(%1)::Any
└──      return %2



### Example: Global scope

A typical cause of type instability are global variables. From a compiler perspective, variables defined in global scope **can change their value and even their type(!) any time**.

In [4]:
a_glob = 2.0
b_glob = 3.0

f() = 2 * a_glob + b_glob

f (generic function with 1 method)

In [5]:
f()

7.0

In [6]:
@code_llvm f()

;  @ /home/javier/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:4 within `f`
define nonnull {}* @julia_f_1356() #0 {
top:
  %0 

= alloca [2 x {}*], align 8
  %gcframe2 = alloca [4 x {}*], align 16
  %gcframe2.sub = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 0
  %.sub = getelementptr inbounds [2 x {}*], [2 x {}*]* %0, i64 0, i64 0
  %1 = bitcast [4 x {}*]* %gcframe2 to i8*
  call void @llvm.memset.p0i8.i64(i8* align 16 %1, i8 0, i64 32, i1 true)
  %thread_ptr = call i8* asm "movq %fs:0, $0", "=r"() #6
  %tls_ppgcstack = getelementptr i8, i8* %thread_ptr, i64 -8
  %2 = bitcast i8* %tls_ppgcstack to {}****
  %tls_pgcstack = load {}***, {}**** %2, align 8
  %3 = bitcast [4 x {}*]* %gcframe2 to i64*
  store i64 8, i64* %3, align 16
  %4 = getelementptr inbounds [4 x {}*], [4 x {}*]* %gcframe2, i64 0, i64 1
  %5 = bitcast {}** %4 to {}***
  %6 = load {}**, {}*** %tls_pgcstack, align 8
  store {}** %6, {}*** %5, align 8
  %7 = bitcast {}*** %tls_pgcstack to {}***
  store {}** %gcframe2.sub, {}*** %7, align 8
  %a_glob = load atomic {}*, {}** inttoptr (i64 140444916446880 to {}**) unordered, ali

In [7]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:4
Arguments
  #self#::Core.Const(f)
Body::Any
1 ─ %1 = (2 * Main.a_glob)::Any
│   %2 = (%1 + Main.b_glob)::Any
└──      return %2



#### Fix 1: Make globals `const`ant

In [8]:
const a_glob_const = 2.0
const b_glob_const = 3.0

f() = 2 * a_glob_const + b_glob_const

f()

7.0

In [9]:
@code_warntype f() # type stable!

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:4
Arguments
  #self#::Core.Const(f)
Body::Float64
1 ─ %1 = (2 * Main.a_glob_const)::Core.Const(4.0)
│   %2 = (%1 + Main.b_glob_const)::Core.Const(7.0)
└──      return %2



In [10]:
@code_llvm f()

;  @ /home/javier/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:4 within `f`
define double @julia_f_1380() #0 {
top:
  ret double 7.000000e+00
}


This is fast. In fact, it's not just fast, but **as fast as it can be**! Julia has figured out the result of the calculation at compile-time.

#### Fix 2: Write self-contained functions

In [11]:
f(a,b) = 2a+b

f (generic function with 2 methods)

In [12]:
@code_warntype f(2.0,3.0)

MethodInstance for f(::Float64, ::Float64)
  from f(a, b) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(f)
  a::Float64
  b::Float64
Body::Float64
1 ─ %1 = (2 * a)::Float64
│   %2 = (%1 + b)::Float64
└──      return %2



## Type inference: Avoid abstract field types

A common reason for type inference to break are **not-concretely typed fields** in `struct`s.

### Example

In [13]:
using BenchmarkTools

In [14]:
struct MyType
    x::Number
    y
end

f(a::MyType) = a.x^2 + sqrt(a.x)

f (generic function with 3 methods)

In [15]:
a = MyType(3.0, "test")

@code_warntype f(a);

MethodInstance for f(::MyType)
  from f(a::MyType) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:6
Arguments
  #self#::Core.Const(f)
  a::MyType
Body::Any
1 ─ %1 = Main.:^::Core.Const(^)
│   %2 = Base.getproperty(a, :x)::Number
│   %3 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %4 = (%3)()::Core.Const(Val{2}())
│   %5 = Base.literal_pow(%1, %2, %4)::Any
│   %6 = Base.getproperty(a, :x)::Number
│   %7 = Main.sqrt(%6)::Any
│   %8 = (%5 + %7)::Any
└──      return %8



In [16]:
@btime f($a);

  40.061 ns (3 allocations: 48 bytes)


In [17]:
typeof(a)

MyType

#### Fix 1: Concrete typing

In [18]:
struct MyTypeConcrete
    x::Float64
    y::String
end

f(b::MyTypeConcrete) = b.x^2 + sqrt(b.x)

f (generic function with 4 methods)

In [19]:
b = MyTypeConcrete(3.0, "test")
@code_warntype f(b)

MethodInstance for f(::MyTypeConcrete)
  from f(b::MyTypeConcrete) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:6
Arguments
  #self#::Core.Const(f)
  b::MyTypeConcrete
Body::Float64
1 ─ %1 = Main.:^::Core.Const(^)
│   %2 = Base.getproperty(b, :x)::Float64
│   %3 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %4 = (%3)()::Core.Const(Val{2}())
│   %5 = Base.literal_pow(%1, %2, %4)::Float64
│   %6 = Base.getproperty(b, :x)::Float64
│   %7 = Main.sqrt(%6)::Float64
│   %8 = (%5 + %7)::Float64
└──      return %8



In [20]:
@btime f($b);

  2.120 ns (0 allocations: 0 bytes)


#### Fix 2: Type parameters

But what if I want to accept any kind of, say, `Number` and `AbstractString` for our type?

In [21]:
struct MyTypeParametric{A<:Number, B<:AbstractString}
    x::A
    y::B
end

f(c::MyTypeParametric) = c.x^2 + sqrt(c.x)

f (generic function with 5 methods)

In [22]:
c = MyTypeParametric(3.0, "test")

MyTypeParametric{Float64, String}(3.0, "test")

In [23]:
@code_warntype f(c)

MethodInstance for f(::MyTypeParametric{Float64, String})
  from f(c::MyTypeParametric) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:6
Arguments
  #self#::Core.Const(f)
  c::MyTypeParametric{Float64, String}
Body::Float64
1 ─ %1 = Main.:^::Core.Const(^)
│   %2 = Base.getproperty(c, :x)::Float64
│   %3 = Core.apply_type(Base.Val, 2)::Core.Const(Val{2})
│   %4 = (%3)()::Core.Const(Val{2}())
│   %5 = Base.literal_pow(%1, %2, %4)::Float64
│   %6 = Base.getproperty(c, :x)::Float64
│   %7 = Main.sqrt(%6)::Float64
│   %8 = (%5 + %7)::Float64
└──      return %8



From the type alone the compiler knows what the structure contains and can produce optimal code:

In [24]:
@btime f($c);

  2.111 ns (0 allocations: 0 bytes)


In [25]:
c = MyTypeParametric(Float32(3.0), SubString("test"))

MyTypeParametric{Float32, SubString{String}}(3.0f0, "test")

In [26]:
@btime f($c);

  2.517 ns (0 allocations: 0 bytes)


## Type inference: Avoid untyped containers

### Example

In [27]:
function f()
    numbers = []
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

  191.103 ns (3 allocations: 464 bytes)


In [28]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(f)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  numbers::Vector{Any}
  i::Int64
Body::Any
1 ─       (numbers = Base.vect())
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::

Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         Main.push!(numbers, i)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄ %16 = Main.sum(numbers)::Any
└──       return %16



In [29]:
typeof([])

Vector{Any} (alias for Array{Any, 1})

In [30]:
function f()
    numbers = Int[]
    for i in 1:10
        push!(numbers, i)
    end
    sum(numbers)
end

@btime f();

  92.301 ns (3 allocations: 480 bytes)


In [31]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(f)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  numbers::Vector{Int64}
  i::Int64
Body::Int64
1 ─       (numbers = Base.getindex(Main.Int))
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         Main.push!(numbers, i)
│         (@_2 = Base.iterate(%2, %9))
│   %12 = (@_2 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄ %16 = Main.sum(numbers)::Int64
└──       return %16



## Type inference: Avoid changing variable types

Variables should not change type.

### Example

In [32]:
function f()
    x = 1
    for i = 1:10
        x /= rand()
    end
    return x
end

f (generic function with 5 methods)

In [33]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(f)
Locals
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  x::Union{Float64, Int64}
  i::Int64
Body::Float64
1 ─       (x = 1)
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = x::Union{Float64, Int64}
│   %11 = Main.rand()::Float64
│         (x = %10 / %11)
│         (@_2 = Base.iterate(%2, %9))
│   %14 = (@_2 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return x::Float64



(On a side note: since the type can only vary between `Float64` and `Int64`, Julia can still produce reasonable code by [*union splitting*](https://julialang.org/blog/2018/08/union-splitting).)

#### Fix: Initialize with correct type

In [34]:
function f()
    x = 1.0
    for i = 1:10
        x /= rand()
    end
    return x
end

f (generic function with 5 methods)

In [35]:
@code_warntype f()

MethodInstance for f()
  from f() @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(f)
Locals
  @_2

::Union{Nothing, Tuple{Int64, Int64}}
  x::Float64
  i::Int64
Body::Float64
1 ─       (x = 1.0)
│   %2  = (1:10)::Core.Const(1:10)
│         (@_2 = Base.iterate(%2))
│   %4  = (@_2::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_2::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = x::Float64
│   %11 = Main.rand()::Float64
│         (x = %10 / %11)
│         (@_2 = Base.iterate(%2, %9))
│   %14 = (@_2 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return x



## Type inference: Isolate unavoidable type instabilities

Type instabilities can occur very naturally, for example when reading unknown user files or user input. Hence, not every instability can be avoided.

If that's the case, isolate your expensive computation from the instability by putting it in a separate *kernel function* (also known as introducing a *function barrier*).

In [36]:
data = [rand((2, 3.4, 5.6f0, "7.8")) for i in 1:100] # random heterogeneous input data

100-element Vector{Any}:
  "7.8"
  "7.8"
 2
 5.6f0
 3.4
 2
 3.4
 2
 2
  "7.8"
 ⋮
 5.6f0
 3.4
 5.6f0
  "7.8"
 2
 3.4
 3.4
  "7.8"
 5.6f0

In [37]:
function expsin_all(data)
    result = zeros(length(data))
    for i in eachindex(data)
        x = data[i]
        if data[i] isa AbstractString
            x = parse(Float64, x)
        end
        result[i] = exp(sin(x))
    end
    return result
end

expsin_all (generic function with 1 method)

In [38]:
@code_warntype expsin_all(data)

MethodInstance for expsin_all(::Vector{Any})
  from expsin_all(data) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(expsin_all)
  data::Vector{Any}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  result::Vector{Float64}
  i::Int64
  x::Any
Body::Vector{Float64}
1 ─ %1  = Main.length(data)::Int64
│         (result = Main.zeros(%1))
│   %3  = Main.eachindex(data)::Base.OneTo{Int64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #6 if not %6
2 ┄ %8  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (x = Base.getindex(data, i))
│   %12 = Base.getindex(data, i)::Any
│   %13 = (%12 isa Main.AbstractString)::Bool
└──       goto #4 if not %13
3 ─       (x = Main.parse(Main.Float64, x))
4 ┄ %16 = Main.sin(x)::Any
│   %17 = Main.exp(%16)::Any
│         Base.setindex!(result, %17, i)
│         (@_3 = Base.iterate

In [39]:
@btime expsin_all($data);

  12.180 μs (227 allocations: 4.41 KiB)


In [40]:
function expsin_all_barrier(data)
    result = zeros(length(data))
    for i in eachindex(data)
        x = data[i]
        result[i] = _expsin_kernel(x)
    end
    return result
end

# function barrier
_expsin_kernel(x::Number)::Float64 = exp(sin(x))
_expsin_kernel(x::AbstractString)::Float64 = exp(sin(parse(Float64, x)))

_expsin_kernel (generic function with 2 methods)

In [41]:
@btime expsin_all_barrier($data);

  4.844 μs (101 allocations: 2.44 KiB)


In [42]:
@code_warntype _expsin_kernel(data[1])

MethodInstance for _expsin_kernel(::String)
  from _expsin_kernel(x::AbstractString) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:12
Arguments
  #self#::Core.Const(_expsin_kernel)
  x::String
Locals
  @_3::Float64
Body::Float64
1 ─ %1 = Main.Float64::Core.Const(Float64)
│   %2 = Main.parse(Main.Float64, x)::Float64
│   %3 = Main.sin(%2)::Float64
│   %4 = Main.exp(%3)::Float64
│        (@_3 = %4)
│   %6 = (@_3 isa %1)::Core.Const(true)
└──      goto #3 if not %6
2 ─      goto #4
3 ─      Core.Const(:(Base.convert(%1, @_3)))


└──      Core.Const(:(@_3 = Core.typeassert(%9, %1)))
4 ┄      return @_3



Note that the computational kernel function is fully type inferred.

In [43]:
@code_warntype expsin_all_barrier(data)

MethodInstance for expsin_all_barrier(::Vector{Any})
  from expsin_all_barrier(data) @ Main ~/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:1
Arguments
  #self#::Core.Const(expsin_all_barrier)
  data::Vector{Any}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  result::Vector{Float64}
  i::Int64
  x::Any
Body::Vector{Float64}
1 ─ %1  = Main.length(data)::Int64
│         (result = Main.zeros(%1))
│   %3  = Main.eachindex(data)::Base.OneTo{Int64}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│         (x = Base.getindex(data, i))
│   %12 = Main._expsin_kernel(x)::Float64
│         Base.setindex!(result, %12, i)
│         (@_3 = Base.iterate(%3, %10))
│   %15 = (@_3 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return resu

## Side note: [JET.jl](JET.jl](https://github.com/aviatesk/JET.jl)

**Static** code analyzer. (Doesn't execute the code!)

Important macros:
* `@report_opt`: check for potential optimization problems ([optimization analysis](https://aviatesk.github.io/JET.jl/stable/optanalysis/))
* `@report_call`: check for potential (general) errors ([error analysis](https://aviatesk.github.io/JET.jl/stable/jetanalysis/))

In [44]:
using JET

In [45]:
@report_opt expsin_all(data)

═════ 7 possible errors found ═════
┌ expsin_all(data::Vector{Any}) @ Main /home/javier/JuliaUCL24/notebooks/Day2/1_perfopt_typestability.ipynb:6
│┌ parse(::Type{Float64}, s::AbstractString) @ Base ./parse.jl:393
││┌ parse(::Type{Float64}, s::AbstractString; kwargs::@Kwargs{}) @ Base ./parse.jl:393
│││┌ tryparse_internal(::Type{Float64}, s::AbstractString, raise::Bool) @ Base ./parse.jl:380
││││┌ tryparse_internal(::Type{Float64}, s::AbstractString, raise::Bool; kwargs::@Kwargs{}) @ Base ./parse.jl:383
│││││┌ _parse_failure(T::Type{Float64}, s::AbstractString) @ Base ./parse.jl:387
││││││┌ lastindex(s::LazyString) @ Base ./strings/lazy.jl:93
│││││││┌ String(l::LazyString) @ Base ./strings/lazy.jl:83
││││││││┌ sprint(::Base.var"#108#109"{LazyString}) @ Base ./strings/io.jl:107
│││││││││┌ sprint(::Base.var"#108#109"{LazyString}; context::Nothing, sizehint::Int64) @ Base ./strings/io.jl:114
││││││││││┌ (::Base.var"#108#109"{LazyString})(io::IOBuffer) @ Base ./strings/lazy.jl:85
││││││││││

# Core messages of this Notebook

* **Wrap code in self-contained functions** in performance critical applications, i.e. avoid global scope.
* Write **type-stable code** (check with `@code_warntype` or `Cthulhu.jl`).
* **Types should always have concrete fields.** If you don't know them in advance, use type parameters.
* Isolate unavoidable type instabilities (function barrier).